## 准备数据，从文本中构建向量 ##

In [1]:
#词到向量的转换函数
def loadDataSet():
    postingList=[
        ['my','dog','has','flea','problem','help','please'],
        ['maybe','not','take','him','to','dog','park','stupid'],
        ['my','dalmation','is','so','cute','I','love','him'],
        ['stop','posting','stupid','worthless','grabage'],
        ['mr','licks','ate','my','steak','how','to','stop','him'],
        ['quit','buyiing','worthless','dog','food','stupid']
    ]
    classVec=[0,1,0,1,0,1]#1表示侮辱性文字 0表示正常文字
    return postingList,classVec
#生成不重复的单词
def createVocabList(dataSet):
    vocabSet=set([])
    #合并
    for document in dataSet:
        vocabSet=vocabSet | set(document)
    return list(vocabSet)
def setOfWords2Vec(vocabList,inputSet):
    returnVec=[0]*len(vocabList)#生成全是0 长度为字典长度的字典
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]=1
        else:
            print('the word %s is not in my vocabulary'%(word))
    return returnVec
listOPosts,listClasses=loadDataSet()
myVocabList=createVocabList(listOPosts)
print(myVocabList)

['him', 'problem', 'please', 'my', 'to', 'so', 'steak', 'mr', 'maybe', 'is', 'food', 'take', 'dalmation', 'ate', 'flea', 'park', 'not', 'stupid', 'posting', 'worthless', 'has', 'stop', 'buyiing', 'help', 'grabage', 'licks', 'quit', 'love', 'dog', 'I', 'how', 'cute']


In [2]:
setOfWords2Vec(myVocabList,listOPosts[0])

[0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0]

## 训练算法，从词向量到计算概率 ##

In [3]:
from numpy import *
#朴素贝叶斯分类训练函数
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs=len(trainMatrix)
    numWords=len(trainMatrix[0])
    pAbusive=sum(trainCategory)/float(numTrainDocs)
    p0Num=zeros(numWords)
    p1Num=zeros(numWords)
    p0Denom=0.0
    p1Denom=0.0
    for i in range(numTrainDocs):
        if trainCategory[i]==1:
            p1Num+=trainMatrix[i]
            p1Denom+=sum(trainMatrix[i])
        else:
            p0Num+=trainMatrix[i]
            p0Denom+=sum(trainMatrix[i])
    p1Vect=p1Num/p1Denom
    p0Vect=p0Num/p0Denom
    return p0Vect,p1Vect,pAbusive

In [4]:
trainMat=[]
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
p0V,p1V,pAb=trainNB0(trainMat,listClasses)
print(pAb)
print(p0V)
print(p1V)

0.5
[0.08333333 0.04166667 0.04166667 0.125      0.04166667 0.04166667
 0.04166667 0.04166667 0.         0.04166667 0.         0.
 0.04166667 0.04166667 0.04166667 0.         0.         0.
 0.         0.         0.04166667 0.04166667 0.         0.04166667
 0.         0.04166667 0.         0.04166667 0.04166667 0.04166667
 0.04166667 0.04166667]
[0.05263158 0.         0.         0.         0.05263158 0.
 0.         0.         0.05263158 0.         0.05263158 0.05263158
 0.         0.         0.         0.05263158 0.05263158 0.15789474
 0.05263158 0.10526316 0.         0.05263158 0.05263158 0.
 0.05263158 0.         0.05263158 0.         0.10526316 0.
 0.         0.        ]


## 测试算法：根据现实情况修改分类器 ##

### 1.极大似然估计，出现0抹去的情况，使用拉普拉斯修正 ###
### 2.连续相乘导致下溢 ###

In [5]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs=len(trainMatrix)
    numWords=len(trainMatrix[0])
    pAbusive=sum(trainCategory)/float(numTrainDocs)
    #使用拉普拉斯修正
    p0Num=ones(numWords)
    p1Num=ones(numWords)
    p0Denom=2.0
    p1Denom=2.0
    for i in range(numTrainDocs):
        if trainCategory[i]==1:
            p1Num+=trainMatrix[i]
            p1Denom+=sum(trainMatrix[i])
        else:
            p0Num+=trainMatrix[i]
            p0Denom+=sum(trainMatrix[i])
    #使用对数防止下溢
    p1Vect=log(p1Num/p1Denom)
    p0Vect=log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive

## 朴素贝叶斯分类函数 ##

In [6]:
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1=sum(vec2Classify*p1Vec)+log(pClass1)
    p0=sum(vec2Classify*p0Vec)+log(1.0-pClass1)
    if p1>p0:
        return 1
    else:
        return 0
def testingNB():
    listOPosts,listClasses=loadDataSet()
    myVocabList=createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0V,p1V,pAb=trainNB0(trainMat,listClasses)
    testEntry=['love','my','dalmation']
    thisDoc=array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classifyed as : ',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry=['stupid','garbage']
    thisDoc=array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as : ',classifyNB(thisDoc,p0V,p1V,pAb))
testingNB()

['love', 'my', 'dalmation'] classifyed as :  0
the word garbage is not in my vocabulary
['stupid', 'garbage'] classified as :  1


##  朴素贝叶斯词袋模型 ##

In [7]:
def bagOfWords2VecMN(vocabList,inputSet):
    returnVec=[0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]+=1
    return returnVec

# 示例：使用朴素贝叶斯过滤垃圾邮件 #

In [8]:
mySent='this book is the best book on python or M.L i have ever laid eyes upon'
mySent.split()


['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'M.L',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

可以看到，切分效果不错，但是标点符号也被当成了词的一部分，可以用正则表示来切分句子，其中分隔符是除单词，数字外的任意字符串

In [9]:
import re
regEx=re.compile('\\W*')
listOfTokens=regEx.split(mySent)
listOfTokens

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: split() requires a non-empty pattern match.
  This is separate from the ipykernel package so we can avoid doing imports until


['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'M',
 'L',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

In [10]:
[tok.lower() for tok in listOfTokens if(len(tok)>0)]

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'm',
 'l',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

In [11]:
emailText=open('email/ham/6.txt').read()
listOfTokens=regEx.split(emailText)

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: split() requires a non-empty pattern match.
  


## 测试：使用朴素贝叶斯进行交叉验证 ##

In [12]:
def spamTest():
    docList=[]; classList = []; fullText =[]
    for i in range(1,26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    trainingSet = list(range(50)); testSet=[]           #create test set
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
            print ("classification error",docList[docIndex])
    print ('the error rate is: ',float(errorCount)/len(testSet))
    #return vocabList,fullText
    

In [13]:
spamTest()

NameError: name 'textParse' is not defined

# 使用朴素贝叶斯分类器从个人广告 #

In [16]:
import feedparser
ny = feedparser.parse('https://newyork.craigslist.org/search/stp?format=rss')
sf = feedparser.parse('https://sfbay.craigslist.org/search/stp?format=rss')
print(ny['entries'])
print(len(ny['entries']))


[]
0
